

# Ethnicity granular mapping using algorithm from ccu069_37 - Sara Khalid's group (Pineda-Moncusi et al, 2024) 


**Adapted for ccu013_02 by Ana Torralbo** 

**Description**  
  
1. Take the latest ethnicity record from each source
2. Create prioritisation to get one code per patient, from:
   a) SNOMED GDPPR code if not null

   b) Primary GDPPR code if not null

   c) Primary HES code if not null


 
**Data input**   
* gdppr_data =  `dars_nic_391419_j3w9t_collab.gdppr_dars_nic_391419_j3w9t_archive` 
* hes_data = `dars_nic_391419_j3w9t_collab.hes_apc_all_years`  
    
**Data output**  
* `dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular` 1 row per patient


## Production dates for GDPPR and HES tables

In [0]:
gdppr_prod_date = "2022-01-20 14:58:52.353312"
hes_prod_date ="2022-01-20 14:58:52.353312"

**0. Load helpers**

In [0]:
%run /Repos/a.torralbo@ucl.ac.uk/ccu013_02/atlas/00_Load_helpers

# 1. Extract Eth using priority ranking

In [0]:
#SNOMED codes, 1 as first choice ranking
spark.sql(f"""CREATE OR REPLACE TEMP VIEW ccu013_02_gdppr_ethnicity_SNOMED AS 
               SELECT NHS_NUMBER_DEID,
                      ConceptId as SNOMED_ETHNIC,
                      PrimaryCode as ETHNIC,
                      `DATE` as RECORD_DATE,
                      'GDPPR_snomed' as dataset,
                      1 as eth_rank,
               recent_rank
               FROM (select *, row_number() over (partition by NHS_NUMBER_DEID order by (CASE WHEN ETHNIC IS NULL or TRIM(ETHNIC) IN ("","9", "99", "X" , "Z") THEN 1 ELSE 0 END), REPORTING_PERIOD_END_DATE desc) as recent_rank 
                FROM dars_nic_391419_j3w9t_collab.gdppr_dars_nic_391419_j3w9t_archive as gdppr
                      INNER JOIN dss_corporate.gdppr_ethnicity_mappings eth on gdppr.`CODE` = eth.ConceptId
                      WHERE gdppr.ProductionDate ==  "2022-01-20 14:58:52.353312"
                )
                WHERE recent_rank = 1 """)

In [0]:
gdppr_sch =spark.sql(""" 
select * from dars_nic_391419_j3w9t.gdppr_dars_nic_391419_j3w9t limit 3""")

gdppr_sch.schema

In [0]:
%sql
SELECT COUNT(*) FROM ccu013_02_gdppr_ethnicity_SNOMED 



In [0]:
%sql
SELECT COUNT(DISTINCT NHS_NUMBER_DEID) FROM ccu013_02_gdppr_ethnicity_SNOMED 


In [0]:
# Primary codes, 2 as second choice ranking
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW ccu013_02_gdppr_ethnicity AS
       SELECT NHS_NUMBER_DEID,
              ETHNIC, REPORTING_PERIOD_END_DATE as RECORD_DATE,
              'GDPPR' as dataset,
              2 as eth_rank,
       recent_rank
       FROM (select *, row_number() over (partition by NHS_NUMBER_DEID order by (CASE WHEN ETHNIC IS NULL or TRIM(ETHNIC) IN ("","9", "99", "X" , "Z")THEN 1 ELSE 0 END), REPORTING_PERIOD_END_DATE desc) as recent_rank
       FROM dars_nic_391419_j3w9t_collab.gdppr_dars_nic_391419_j3w9t_archive as gdppr
       WHERE gdppr.ProductionDate ==  "2022-01-20 14:58:52.353312")
       WHERE recent_rank = 1""")

In [0]:
spark.sql(f"""
 CREATE OR REPLACE TEMP VIEW ccu013_02_all_hes_apc_ethnicity AS
 SELECT DISTINCT PERSON_ID_DEID as NHS_NUMBER_DEID, 
      ETHNOS as ETHNIC, 
      EPISTART as RECORD_DATE, 
      "hes_apc" as dataset,
      3 as eth_rank,
      recent_rank
  FROM (select *, row_number() over (partition by PERSON_ID_DEID order by (CASE WHEN ETHNOS IS NULL or TRIM(ETHNOS) IN ("","9", "99", "X" , "Z") THEN 1 ELSE 0 END), EPISTART desc) as recent_rank from dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive WHERE ProductionDate == '2022-01-20 14:58:52.353312')
  WHERE recent_rank = 1""")

In [0]:
%sql
SELECT COUNT(*) FROM ccu013_02_all_hes_apc_ethnicity


In [0]:
%sql
SELECT COUNT(DISTINCT NHS_NUMBER_DEID) FROM ccu013_02_all_hes_apc_ethnicity


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW ccu013_02_ethnicity_all_codes_latest as
SELECT NHS_NUMBER_DEID, ETHNIC, NULL AS SNOMED_ETHNIC, RECORD_DATE, dataset, eth_rank 
FROM ccu013_02_gdppr_ethnicity
UNION ALL
SELECT NHS_NUMBER_DEID, ETHNIC, SNOMED_ETHNIC, RECORD_DATE, dataset, eth_rank 
FROM ccu013_02_gdppr_ethnicity_SNOMED
UNION ALL
SELECT NHS_NUMBER_DEID, ETHNIC, null AS SNOMED_ETHNIC, RECORD_DATE, dataset, eth_rank 
FROM ccu013_02_all_hes_apc_ethnicity

In [0]:
%sql
-- Flagging the null data in the latest codes
CREATE OR REPLACE TEMP VIEW ccu013_02_ethnicity_all_codes_latest_null as
SELECT *,
     CASE WHEN ETHNIC IS NULL or TRIM(ETHNIC) IN ("","9", "99", "X" , "Z") THEN 1 ELSE 0 END as ethnic_null,
     CASE WHEN SNOMED_ETHNIC IS NULL THEN 1 ELSE 0 END as SNOMED_ethnic_null
     FROM ccu013_02_ethnicity_all_codes_latest

In [0]:
%sql
-- Adding in ethnicity_rank
CREATE OR REPLACE TEMP VIEW ccu013_02_ethnicity_all_codes_latest_null_rank as
SELECT *,
     CASE
         WHEN SNOMED_ethnic_null = 0 THEN 3 
         WHEN (eth_rank = 2 AND ethnic_null = 0) THEN 2
         WHEN (eth_rank = 3 AND ethnic_null = 0) THEN 1
         ELSE 0
       END as ethnicity_rank
     FROM ccu013_02_ethnicity_all_codes_latest_null

In [0]:
spark.sql(f"""
 CREATE OR REPLACE TEMP VIEW ccu013_02_all_hes_apc_ethnicity AS
 SELECT DISTINCT PERSON_ID_DEID as NHS_NUMBER_DEID, 
      ETHNOS as ETHNIC, 
      EPISTART as RECORD_DATE, 
      "hes_apc" as dataset,
      3 as eth_rank,
      recent_rank
  FROM (select *, row_number() over (partition by PERSON_ID_DEID order by (CASE WHEN ETHNOS IS NULL or TRIM(ETHNOS) IN ("","9", "99", "X" , "Z") THEN 1 ELSE 0 END), EPISTART desc) as recent_rank from dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive WHERE ProductionDate == '2022-01-20 14:58:52.353312')
  WHERE recent_rank = 1""")

In [0]:
spark.sql(f"""
 CREATE OR REPLACE TEMP VIEW ccu013_02_ethnicity_assembled_all_codes AS
 SELECT DISTINCT NHS_NUMBER_DEID, 
      ETHNIC,
      SNOMED_ETHNIC,
      RECORD_DATE, 
      dataset,
      ranking
  FROM (select *, row_number() over (partition by NHS_NUMBER_DEID order by ethnicity_rank desc) as ranking from ccu013_02_ethnicity_all_codes_latest_null_rank) 
       WHERE ranking = 1""")

In [0]:
# Adding in 11 levels of granularity (as per ONS) 
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW ccu013_02_ethnicity_assembled_multiple_granularities AS
SELECT 
NHS_NUMBER_DEID,
CASE WHEN ETHNIC IN ('1','2','3','N','M','P') THEN "Black or Black British"
           WHEN ETHNIC IN ('0','A','B','C', 'T') THEN "White"
           WHEN ETHNIC IN ('4','5','6','L','K','J','H', 'R') THEN "Asian or Asian British"
           WHEN ETHNIC IN ('7','8','W','S') THEN "Other Ethnic Group"
           WHEN ETHNIC IN ('D','E','F','G') THEN "Mixed"
           WHEN ETHNIC IN ("9", "99", "X" , "Z") THEN "Unknown"
           ELSE 'Unknown' END as ethnicity_5_group,           
CASE WHEN ETHNIC IN ("1", "M") THEN "Black Caribbean"
           WHEN ETHNIC IN ("2", "N") THEN "Black African"
           WHEN ETHNIC IN ("3", "P") THEN "Other Black"
           WHEN ETHNIC IN ("4", "H") THEN "Indian"
           WHEN ETHNIC IN ("5", "J") THEN "Pakistani"
           WHEN ETHNIC IN ("6", "K") THEN "Bangladeshi"
           WHEN ETHNIC = "L" THEN "Other Asian"
           WHEN ETHNIC IN ('D','E','F','G') THEN "Mixed"
           WHEN ETHNIC IN ("7", "R") THEN "Chinese"
           WHEN ETHNIC IN ('0','A','B','C','T') THEN "White"
           WHEN ETHNIC IN ("8", 'S', "W") THEN "Other Ethnic Group"
           WHEN ETHNIC IN ("9","99","X","Z") THEN "Unknown"
           ELSE 'Unknown' END as ethnicity_11_group,
ETHNIC as PrimaryCode_ethnicity,           
SNOMED_ETHNIC as SNOMED_ethnicity,           
RECORD_DATE, 
dataset 
FROM ccu013_02_ethnicity_assembled_all_codes """)

In [0]:
%sql 
select * from ccu013_02_ethnicity_assembled_multiple_granularities limit 4;

In [0]:
assembledethn_ccu013_02 = spark.sql(f'''
SELECT * 
FROM ccu013_02_ethnicity_assembled_multiple_granularities e
WHERE e.NHS_NUMBER_DEID in
 (SELECT s.NHS_NUMBER_DEID 
 from dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort s)
 ''')

# 2. Export and check

In [0]:
#drop_table("dsa_391419_j3w9t_collab.ccu069_37_ethnicity_assembled_multiple_granularities")

assembledethn_ccu013_02.write.mode("overwrite").saveAsTable("dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate")

In [0]:
%sql
SELECT * FROM dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate limit 10;

In [0]:
%sql 
select count(distinct(NHS_NUMBER_DEID)) 
FROM dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate;

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate;

In [0]:
%sql 
select distinct(ethnicity_11_group) from dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate;

In [0]:
%sql 
select count(distinct(NHS_NUMBER_DEID)) as n_indiv, ethnicity_5_group from dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate 
group by ethnicity_5_group;

In [0]:
%sql 
select count(distinct(NHS_NUMBER_DEID)) as n_indiv, ethnicity_11_group from dsa_391419_j3w9t_collab.ccu013_02_ethnicity_granular_ProdDate 
group by ethnicity_11_group;